## [Repast for Python (Repast4Py) User Guide](https://repast.github.io/repast4py.site/guide/user_guide.html)

## [API](https://repast.github.io/repast4py.site/apidoc/index.html)

## [GitHub Repast/repast4py](https://github.com/Repast/repast4py)

## [MPI for Python](https://mpi4py.readthedocs.io/en/stable/tutorial.html#collective-communication)

# An idea for the initial example: 

### something close to the [Chakraborti model](https://www.scientificamerican.com/article/is-inequality-inevitable/) of Winners, Losers 

ESC L set or unset row numbers

the script below avoids scrolling outputs

In [1]:
%%javascript
// to avoid scroll in windows
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

## 0

program starter

In [2]:
%%writefile starter.py

from repast4py import parameters
from runMod import *

# https://repast.github.io/repast4py.site/apidoc/source/repast4py.parameters.html
"""
create_args_parser()
Creates an argparse parser with two arguments: 
1) a yaml format file containing model parameter input, and 
2) an optional json dictionary string that can override that input.
"""
parser = parameters.create_args_parser()

args = parser.parse_args()
#print(1,args,flush=True)

"""
init_params(parameters_file, parameters)
Initializes the repast4py.parameters.params dictionary with the model input parameters.
"""
params = parameters.init_params(args.parameters_file, args.parameters)
#print(2,params,flush=True)

run(params)

Overwriting starter.py


## 1 

run the model

In [3]:
%%writefile runMod.py

from typing import Dict
from model import *

def run(params: Dict):
    #print(3, params)
    #print (4, params['random.seed'])
    #print (4.1, params['a'])
    
    model = Model(MPI.COMM_WORLD, params)
    model.start()

Overwriting runMod.py


## 2

agent definition

In [4]:
%%writefile WinnerLoser.py

from repast4py import core
from typing import Tuple
from repast4py import context as ctx

class WinnerLoser(core.Agent):

    TYPE = 0
    
    def __init__(self, local_id: int, rank: int, wallet: float):
        super().__init__(id=local_id, type=WinnerLoser.TYPE, rank=rank)

        self.myWallet = wallet
        
    def lookForMinWallet(self,agSet):

        agWalletSet=list(aWinnerLoser.myWallet for aWinnerLoser in agSet)
        self.minWalletPosition=agWalletSet.index(min(agWalletSet))
        
    def give(self,agSet):
        
        list(agSet)[self.minWalletPosition].myWallet+=1
        self.myWallet-=1
        
    def save(self) -> Tuple:
        """
        Saves the state of the WinnerLoser as a Tuple.

        Returns:
            The saved state of this Walker.
        """
        return (self.uid, self.myWallet)


        
class Ghostbuster(core.Agent):

    TYPE = 1
    
    def __init__(self, local_id: int, rank: int, context: ctx, prey: Tuple):
        super().__init__(id=local_id, type=Ghostbuster.TYPE, rank=rank)
        
        self.myPrey=prey
        self.context=context
        
    def search(self,tick):
        print("tick",tick,"ghostbuster",self.uid, "searching",self.myPrey,flush=True)
        """
        https://repast.github.io/repast4py.site/apidoc/source/repast4py.context.html
        agent(agent_id)
        Gets the specified agent from the collection of local agents in this context.

        Parameters
        agent_id – the unique id tuple of the agent to return

        Returns
        The agent with the specified id or None if no such agent is found.

        Return type
        _core.Agent
        """
        if self.context.agent(self.myPrey) == None: 
            print("Not found in my rank",flush=True)
            
            """
            https://repast.github.io/repast4py.site/apidoc/source/repast4py.context.html
            request_agents(requested_agents, create_agent)
            Requests agents from other ranks to be copied to this rank as ghosts.

            This is a collective operation and all ranks must call it, regardless 
            of whether agents are being requested by that rank. The requested agents 
            will be automatically added as ghosts to this rank.

            Parameters
            requested_agents (List) – A list of tuples specifying requested 
            agents and the rank to request from. Each tuple must contain the agents 
            unique id tuple and the rank, for example ((id, type, rank), requested_rank).

            create_agent (Callable) – a Callable that can take the result of an agent 
            save() and return an agent.

            Returns
            The list of requested agents.

            Return type
            List[_core.Agent]
            """
            
            a=self.context.request_agents([(self.myPrey,self.myPrey[2])])
            #print("Ghost created",a,flush=True)
        

Overwriting WinnerLoser.py


## 3

the model

In [5]:
%%writefile model.py

from mpi4py import MPI
from typing import Dict
from repast4py import schedule
from repast4py import context as ctx
import repast4py
import time
"""
time() -> floating point number
        Return the current time in seconds since the Epoch.
        Fractions of a second may be present if the system clock provides them.
to know the Epoch, time.gmtime(0) (in Unix: 19700101)
"""
from WinnerLoser import *

class Model:
    """
    The Model class encapsulates the simulation, and is
    responsible for initialization (scheduling events, creating agents,
    and the grid the agents inhabit IF ANY), and the overall iterating
    behavior of the model.

    Args:
        comm: the mpi communicator over which the model is distributed.
        params: the simulation input parameters
    """
    def __init__(self, comm: MPI.Intracomm, params: Dict):

        self.rank    = comm.Get_rank()
        self.rankNum = comm.Get_size() #pt
        
        #print(5, "rank", self.rank, "rank number", self.rankNum)
        
        
        # create the context to hold the agents and manage cross process
        # synchronization
        self.context = ctx.SharedContext(comm)
        
        # create the schedule
        # https://repast.github.io/repast4py.site/apidoc/source/repast4py.schedule.html
                
        """
        init_schedule_runner(comm)
        Initializes the default schedule runner, a dynamic schedule of executable 
        events shared and synchronized across processes.
        Events are added to the scheduled for execution at a particular tick. 
        The first valid tick is 0. Events will be executed in tick order, earliest 
        before latest. Events scheduled for the same tick will be executed in the 
        order in which they were added. If during the execution of a tick, 
        an event is scheduled before the executing tick (i.e., scheduled to occur in 
        the past) then that event is ignored. The scheduled is synchronized across 
        process ranks by determining the global cross-process minimum next scheduled 
        event time, and executing only the events schedule for that time. In this way, 
        no schedule runs ahead of any other.
        """
        self.runner = schedule.init_schedule_runner(comm)
        
        """
        schedule_repeating_event(at, interval, evt)
        Schedules the specified event to execute at the specified tick, and repeat at 
        the specified interval.

        Parameters
        at (float) – the time of the event.
        interval (float) – the interval at which to repeat event execution.
        evt (Callable) – the Callable to execute when the event occurs.

            A callable is anything that can be called.
            The built-in callable (PyCallable_Check in objects.c) checks if the argument 
            is either:
                an instance of a class with a __call__ method or
                is of a type that has a non null tp_call (c struct) member which 
                indicates callability otherwise (such as in functions, methods etc.)
        """
        self.runner.schedule_repeating_event(0, 1, self.lookAtWalletsAndGive)
        self.runner.schedule_repeating_event(0.1, 1, self.activateGhostbusters)
        
        """
        schedule_stop(at)
        Schedules the execution of this schedule to stop at the specified tick.

        Parameters
        at (float) – the tick at which the schedule will stop.
        """
        self.runner.schedule_stop(params['stop.at'])
        
        self.runner.schedule_end_event(self.finish)
        

        
        # create agents
        
        # https://repast.github.io/repast4py.site/apidoc/source/repast4py.random.html
        """
        Random numbers for repast4py. When this module is imported, 
        repast4py.random.default_rng is created using the current epoch time as 
        the random seed, and repast4py.random.seed is set to that value. 
        
        repast4py.random.init(rng_seed=None)
        Initializes the default random number generator using the specified seed.
        """
        repast4py.random.init(rng_seed=params['myRandom.seed'][self.rank])
        rng = repast4py.random.default_rng 
        
        # winnerLoser agents
        
        for i in range(params['WinnerLoser.count'] // self.rankNum): 
                                                #to subdivide the total #pt
            # create and add the agent to the context
            aWallet=10 * rng.random()
            #print(aWallet,flush=True)
            aWinnerLoser = WinnerLoser(i,self.rank,aWallet)
            self.context.add(aWinnerLoser)
            
        # ghostbuster agents
  
        if self.rank==0:
            aGhostbuster = Ghostbuster(0,self.rank,self.context,(0,0,1))
            self.context.add(aGhostbuster) 
            
        
    def lookAtWalletsAndGive(self):
        
        """
        NOT NECESARY IF NO PROJECTIONS
        synchronize(restore_agent, sync_ghosts=True)
        Synchronizes the model state across processes by moving agents, 
        filling projection buffers with ghosts, updating ghosted state and so forth.

        Parameters
        restore_agent (Callable) – a callable that takes agent state data and 
        returns an agent instance from that data. The data is a tuple whose first 
        element is the agent’s unique id tuple, and the second element is the 
        agent’s state, as returned by that agent’s type’s save() method.

        sync_ghosts (bool) – if True, the ghosts in any SharedProjections and 
        value layers associated with this SharedContext are also synchronized. 
        Defaults to True.
        """
                
        """
        agents(agent_type=None, count=None, shuffle=False)
        Gets the agents in this SharedContext, optionally of the specified type, count 
        or shuffled.

        Parameters
        agent_type (int) – the type id of the agent, defaults to None.
        count (int) – the number of agents to return, defaults to None, meaning return 
        all the agents.shuffle (bool) – whether or not the iteration order is shuffled.
        If true, the order is shuffled. If false, the iteration order is the order of 
        insertion.

        Returns
        An iterable over all the agents in the context. If the agent_type is 
        not None then an iterable over agents of that type will be returned.

        Return type
        iterable 
        pt addendum: it is a generator, not a list
        """

        tick = self.runner.schedule.tick        
        print("rank",self.rank,"at tick",tick,"clock",time.time(),\
                            "\nwallets",list(aWinnerLoser.myWallet\
                            for aWinnerLoser in self.context.agents(agent_type=0)),\
                            flush=True)
                
        for aWinnerLoser in self.context.agents(agent_type=0):
            aWinnerLoser.lookForMinWallet(self.context.agents(agent_type=0))
            
            aWinnerLoser.give(self.context.agents(agent_type=0))
            
    def activateGhostbusters(self):
        tick = self.runner.schedule.tick

        """
        https://repast.github.io/repast4py.site/apidoc/source/repast4py.context.html
        
        size(agent_type_ids=None)
        Gets the number of local agents in this SharedContext, optionally by type.

        Parameters
        agent_type_ids (List[int]) – a list of the agent type ids identifying the 
        agent types to count. If this is None then the total size is returned with 
        an agent type id of -1.

        Returns
        A dictionary containing the counts (the dict values) by agent type (the 
        dict keys).

        Return type
        dict

"""
        try: n=self.context.size([1])[1]
        except: n=0
        if n>0:
            for aGhostbuster in self.context.agents(agent_type=1):  
                #self.context.agents(agent_type=1) gives na error if no agent od type 1
                # exist
                aGhostbuster.search(tick)
                
    def finish(self):
        tick = self.runner.schedule.tick
        print("ciao by rank",self.rank,"at tick",tick,"clock",time.time(),flush=True)
        
    def start(self):
        self.runner.execute()
        

Overwriting model.py


# Execution

In [6]:
# R U N
# %sx mpirun -n 1 python3 starter.py winners-losers.yaml
! mpirun -n 3 python3 starter.py winners-losers.yaml
# %sx mpirun -n 1 python3 starter.py winners-losers.yaml '{"a": 123,"random.seed": 1000}'

rank 0 at tick 0 clock 1664195586.139287 
wallets [7.774267584595207, 9.447334544083937, 9.892916439283583, 4.108857789574312]
rank 1 at tick 0 clock 1664195586.1392908 
wallets [1.8225921321259864, 2.0367613231682116, 2.702422668047857, 8.375666934062558]
rank 2 at tick 0 clock 1664195586.1393101 
wallets [6.18864376390266, 6.875605504366426, 4.157527840324256, 2.4272903512617408]
tick 0.1 ghostbuster (0, 1, 0) searching (0, 0, 1)
Not found in my rank
rank 0 at tick 1 clock 1664195586.139719 
wallets [7.774267584595207, 8.447334544083937, 8.892916439283583, 6.108857789574312]
rank 1 at tick 1 clock 1664195586.1397278 
wallets [2.8225921321259864, 2.0367613231682116, 2.702422668047857, 7.375666934062558]
rank 2 at tick 1 clock 1664195586.139732 
wallets [5.18864376390266, 5.875605504366426, 5.157527840324256, 3.4272903512617408]
tick 1.1 ghostbuster (0, 1, 0) searching (0, 0, 1)
Not found in my rank
rank 0 at tick 2 clock 1664195586.139957 
wallets [7.774267584595207, 8.447334544083937